In [1]:
# Standard imports
import pandas as pd
import numpy as np
import os
import sys

# Append the project root directory to the system path to import modules
# The path is relative to the notebook's location in 'notebooks/'
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Import custom modules
from src.loader import DataLoader
from src.eda_utils import EDAPlotter

print("Custom modules (DataLoader, EDAPlotter) imported successfully.")

Custom modules (DataLoader, EDAPlotter) imported successfully.


In [25]:
# Instantiate and Load Data (The DVC-tracked file)
loader = DataLoader('../data/raw/insurance_claims.csv')
df = loader.load_data()
df = loader.clean_column_names()
df = loader.optimize_dtypes()

print(f"Data successfully loaded. Shape: {df.shape}")
print("\n--- Data Quality Check ---")
# Print a concise summary of column types and memory usage
df.info(verbose=False, memory_usage="deep")
print("\nTop 5 Missing Columns:")
# Identify critical missing data points for your interim report
print(df.isnull().sum().sort_values(ascending=False).head())

Data successfully loaded. Shape: (1000098, 52)

--- Data Quality Check ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000098 entries, 0 to 1000097
Columns: 52 entries, underwrittencoverid to totalclaims
dtypes: bool(1), category(36), float64(11), int64(4)
memory usage: 152.7 MB

Top 5 Missing Columns:
numberofvehiclesinfleet    1000098
crossborder                 999400
customvalueestimate         779642
rebuilt                     641901
converted                   641901
dtype: int64


In [26]:
# In Cell 3: Feature Engineering (The KPI Metric)

# CRITICAL FIX: Ensure 'transactionmonth' is the correct datetime type, overriding any previous type issues.
if 'transactionmonth' in df.columns:
    # Coercing errors will turn invalid date strings into NaT (Not a Time)
    df['transactionmonth'] = pd.to_datetime(df['transactionmonth'], errors='coerce')
    # Drop any rows where the date failed to parse (safeguards against resample issues)
    df.dropna(subset=['transactionmonth'], inplace=True)
    
    # DIAGNOSTIC CHECK: Print the type to confirm the fix
    print(f"TransactionMonth Dtype after fix: {df['transactionmonth'].dtype}")


# Calculate the industry-standard KPI: LOSS RATIO
# Loss Ratio = Total Claims / Total Premium
df['loss_ratio'] = df['totalclaims'] / df['totalpremium']

# CRITICAL DATA CLEANING: Handle division by zero/NaN/Inf values.
df['loss_ratio'].replace([np.inf, -np.inf], np.nan, inplace=True)
df['loss_ratio'] = df['loss_ratio'].fillna(0) 

# Cap the loss ratio for robust visualization. Loss Ratio > 1.0 means unprofitable.
df['capped_loss_ratio'] = df['loss_ratio'].clip(upper=5.0) 

print(f"\nLoss Ratio calculated.")
print(f"Mean Loss Ratio (Profitability Indicator): {df['loss_ratio'].mean():.4f}")
print(f"Max Capped Loss Ratio: {df['capped_loss_ratio'].max():.2f}")

TransactionMonth Dtype after fix: category

Loss Ratio calculated.
Mean Loss Ratio (Profitability Indicator): 0.2164
Max Capped Loss Ratio: 5.00


/tmp/ipykernel_14773/3005280302.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['loss_ratio'].replace([np.inf, -np.inf], np.nan, inplace=True)


In [27]:
plotter = EDAPlotter(df)

# Check distribution of the core money variables
plotter.plot_univariate_distribution(column='totalpremium', title_suffix="($) - Skewed")
plotter.plot_univariate_distribution(column='totalclaims', title_suffix="($) - Heavy Right Tail")

# Plot the engineered KPI metric
plotter.plot_univariate_distribution(column='capped_loss_ratio', title_suffix=" (Capped at 5.0)")

print("Cell 4: Univariate plots generated and saved to reports/figures/")

Cell 4: Univariate plots generated and saved to reports/figures/


In [28]:
# Insight 1: Geography (Province) vs. Profitability (Loss Ratio)
plotter.plot_risk_by_category(x_col='province', y_col='loss_ratio', sort=True)

# Insight 2: Vehicle Body Type vs. Claim Severity (totalclaims)
plotter.plot_risk_by_category(x_col='bodytype', y_col='totalclaims', sort=True)

# Insight 3: Demographics (Marital Status) vs. Loss Ratio
plotter.plot_risk_by_category(x_col='maritalstatus', y_col='loss_ratio', sort=True)

print("Cell 5: Bivariate plots generated and saved to reports/figures/")

/home/sovereign/AlphaCare_Insurance_Analytics/src/eda_utils.py:27: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  risk_df = self.df.groupby(x_col)[y_col].mean().reset_index()
/home/sovereign/AlphaCare_Insurance_Analytics/src/eda_utils.py:27: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  risk_df = self.df.groupby(x_col)[y_col].mean().reset_index()


Cell 5: Bivariate plots generated and saved to reports/figures/


/home/sovereign/AlphaCare_Insurance_Analytics/src/eda_utils.py:27: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  risk_df = self.df.groupby(x_col)[y_col].mean().reset_index()


In [32]:
# In Cell 6: Correlation and Time Series - FINAL SOLUTION

# 6.1 Correlation Matrix
numerical_cols = [
    'totalpremium', 
    'totalclaims', 
    'loss_ratio', 
    'customvalueestimate', 
    'registrationyear',    
    'cylinders',           
    'termfrequency'        
]
plotter.plot_correlation_heatmap(numerical_cols)

# 6.2 Time Series of Claims (Creative Plotting)
# FINAL SOLUTION: Convert categorical datetime to regular datetime

# Method 1: Extract datetime from categorical (most reliable)
# Since the categories are already datetime64[ns], we can convert them directly
try:
    # Extract the datetime values from the categorical
    # Convert categorical to its underlying datetime categories
    datetime_values = df['transactionmonth'].cat.categories[df['transactionmonth'].cat.codes]
    
    # Create a new DataFrame with proper datetime
    df_time = df.copy()
    df_time['transactionmonth'] = pd.DatetimeIndex(datetime_values)
    
    # Drop any NaT values (just in case)
    df_time = df_time.dropna(subset=['transactionmonth'])
    
    # Set index and resample - use 'ME' (Month End) instead of deprecated 'M'
    time_series_df = df_time.set_index('transactionmonth').resample('ME')['totalclaims'].sum().reset_index()
    
    print("✓ Time series created using categorical datetime extraction")
    
except Exception as e:
    print(f"Method 1 failed: {e}")
    
    # Method 2: Fallback - convert to string then to datetime
    df_time = df.copy()
    # Convert categorical to string then to datetime
    df_time['transactionmonth'] = pd.to_datetime(df_time['transactionmonth'].astype(str), errors='coerce')
    df_time = df_time.dropna(subset=['transactionmonth'])
    
    # Set index and resample
    time_series_df = df_time.set_index('transactionmonth').resample('ME')['totalclaims'].sum().reset_index()
    
    print("✓ Time series created using string conversion fallback")

# Generate the plot
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid", palette="viridis")

plt.figure(figsize=(14, 6))
sns.lineplot(x='transactionmonth', y='totalclaims', data=time_series_df, marker='o', color=sns.color_palette("viridis")[3])
plt.title('Monthly Total Claims Over Time', fontsize=16)
plt.xlabel("Month", fontsize=12)
plt.ylabel("Total Claims (USD)", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig("../reports/figures/claims_time_series.png")
plt.close()

print("✓ Time series plot generated and saved to reports/figures/claims_time_series.png")
print(f"✓ Data points: {len(time_series_df)} months from {time_series_df['transactionmonth'].min()} to {time_series_df['transactionmonth'].max()}")

✓ Time series created using categorical datetime extraction
✓ Time series plot generated and saved to reports/figures/claims_time_series.png
✓ Data points: 23 months from 2013-10-31 00:00:00 to 2015-08-31 00:00:00


In [41]:
# COMPLETE VISUALIZATION AND TABLE GENERATION - FINAL VERSION
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import os

# Create directories if they don't exist
os.makedirs('../reports/figures', exist_ok=True)
os.makedirs('../reports/tables', exist_ok=True)

print("Generating all missing visualizations and tables...\n")

# 1. Generate Figure 3: Province Risk Distribution
print("1. Generating Province Risk Distribution...")
try:
    # Use observed=False to handle categorical grouping
    grouped = df.groupby('province', observed=False)['loss_ratio'].agg(['mean', 'count', 'std']).reset_index()
    grouped = grouped[grouped['count'] >= 10]  # At least 10 samples
    grouped = grouped.sort_values('mean', ascending=False).head(10)
    
    fig, ax = plt.subplots(figsize=(14, 8))
    colors = sns.color_palette("RdYlBu_r", len(grouped))
    bars = ax.bar(range(len(grouped)), grouped['mean'], color=colors, edgecolor='black')
    
    ax.set_title('Loss Ratio Distribution by Province (Top 10)', fontsize=16, fontweight='bold', pad=20)
    ax.set_xlabel('Province', fontsize=12)
    ax.set_ylabel('Mean Loss Ratio', fontsize=12)
    ax.set_xticks(range(len(grouped)))
    ax.set_xticklabels(grouped['province'], rotation=45, ha='right', fontsize=10)
    
    # Add value labels
    for i, (mean_val, count_val) in enumerate(zip(grouped['mean'], grouped['count'])):
        ax.text(i, mean_val + (0.02 * max(grouped['mean'])), 
                f'{mean_val:.3f}\n(n={count_val:,})', 
                ha='center', va='bottom', fontsize=9)
    
    # Add horizontal line for overall mean
    overall_mean = df['loss_ratio'].mean()
    ax.axhline(y=overall_mean, color='red', linestyle='--', alpha=0.7, 
               label=f'Overall Mean: {overall_mean:.3f}')
    ax.legend()
    ax.grid(axis='y', linestyle='--', alpha=0.3)
    plt.tight_layout()
    plt.savefig('../reports/figures/province_risk_dist.png', dpi=300, bbox_inches='tight')
    plt.close()
    print(f"   ✓ Saved to ../reports/figures/province_risk_dist.png")
except Exception as e:
    print(f"   ✗ Error: {e}")

# 2. Generate Figure 4: Vehicle Type Risk
print("\n2. Generating Vehicle Type Risk Distribution...")
try:
    grouped = df.groupby('vehicletype', observed=False)['loss_ratio'].agg(['mean', 'count', 'std']).reset_index()
    grouped = grouped[grouped['count'] >= 10]
    grouped = grouped.sort_values('mean', ascending=False).head(8)
    
    fig, ax = plt.subplots(figsize=(14, 8))
    colors = sns.color_palette("viridis", len(grouped))
    bars = ax.bar(range(len(grouped)), grouped['mean'], color=colors, edgecolor='black')
    
    ax.set_title('Loss Ratio by Vehicle Type (Top 8)', fontsize=16, fontweight='bold', pad=20)
    ax.set_xlabel('Vehicle Type', fontsize=12)
    ax.set_ylabel('Mean Loss Ratio', fontsize=12)
    ax.set_xticks(range(len(grouped)))
    ax.set_xticklabels(grouped['vehicletype'], rotation=45, ha='right', fontsize=10)
    
    for i, (mean_val, count_val) in enumerate(zip(grouped['mean'], grouped['count'])):
        ax.text(i, mean_val + (0.02 * max(grouped['mean'])), 
                f'{mean_val:.3f}\n(n={count_val:,})', 
                ha='center', va='bottom', fontsize=9)
    
    overall_mean = df['loss_ratio'].mean()
    ax.axhline(y=overall_mean, color='red', linestyle='--', alpha=0.7, 
               label=f'Overall Mean: {overall_mean:.3f}')
    ax.legend()
    ax.grid(axis='y', linestyle='--', alpha=0.3)
    plt.tight_layout()
    plt.savefig('../reports/figures/vehicle_type_risk.png', dpi=300, bbox_inches='tight')
    plt.close()
    print(f"   ✓ Saved to ../reports/figures/vehicle_type_risk.png")
except Exception as e:
    print(f"   ✗ Error: {e}")

# 3. Generate Correlation Heatmap with only numeric columns
print("\n3. Generating Correlation Heatmap...")
try:
    # Only use truly numeric columns
    numeric_cols = ['totalpremium', 'totalclaims', 'loss_ratio', 'customvalueestimate', 
                    'registrationyear', 'cylinders', 'cubiccapacity', 'kilowatts']
    
    # Filter to columns that exist and are numeric
    available_numeric_cols = [col for col in numeric_cols if col in df.columns]
    
    # Check data types to ensure they're numeric
    for col in available_numeric_cols:
        if not pd.api.types.is_numeric_dtype(df[col]):
            print(f"   Warning: {col} is not numeric, removing from correlation")
            available_numeric_cols.remove(col)
    
    if len(available_numeric_cols) >= 2:
        corr = df[available_numeric_cols].corr(numeric_only=True)
        
        plt.figure(figsize=(12, 10))
        mask = np.triu(np.ones_like(corr, dtype=bool))
        sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f", 
                    linewidths=.5, linecolor='black', mask=mask,
                    cbar_kws={"shrink": .8})
        plt.title('Feature Correlation Heatmap', fontsize=16, fontweight='bold', pad=20)
        plt.tight_layout()
        plt.savefig('../reports/figures/correlation_heatmap.png', dpi=300, bbox_inches='tight')
        plt.close()
        print(f"   ✓ Saved to ../reports/figures/correlation_heatmap.png")
    else:
        print("   ✗ Not enough numeric columns for correlation matrix")
except Exception as e:
    print(f"   ✗ Error generating correlation heatmap: {e}")

# 4. GENERATE ALL STATISTICAL TABLES
print("\n4. Generating Statistical Tables...")

# Table 1: Descriptive Statistics
print("\n" + "="*80)
print("TABLE 1: Descriptive Statistics of Core Financial Metrics")
print("="*80)
stats_df = df[['totalpremium', 'totalclaims', 'loss_ratio']].describe().round(2)
print(stats_df.to_markdown())
stats_df.to_csv('../reports/tables/descriptive_stats.csv')
print("\n✓ Saved to reports/tables/descriptive_stats.csv")

# Table 2: Top 5 Feature Correlations with Loss Ratio
print("\n" + "="*80)
print("TABLE 2: Feature Correlations with Loss Ratio")
print("="*80)

# Calculate correlation with loss_ratio for all numeric columns
numeric_features = df.select_dtypes(include=[np.number]).columns.tolist()
if 'loss_ratio' in numeric_features:
    numeric_features.remove('loss_ratio')
    
if numeric_features:
    # Calculate correlations
    correlations = {}
    for feature in numeric_features[:10]:  # Limit to first 10 features for readability
        if df[feature].notna().sum() > 100:  # Only if we have enough data
            corr_value = df['loss_ratio'].corr(df[feature])
            correlations[feature] = corr_value
    
    # Sort by absolute correlation
    sorted_corr = sorted(correlations.items(), key=lambda x: abs(x[1]), reverse=True)[:5]
    
    # Create table
    corr_table_data = []
    for feature, corr_value in sorted_corr:
        strength = "Strong" if abs(corr_value) > 0.5 else "Moderate" if abs(corr_value) > 0.3 else "Weak"
        direction = "Positive" if corr_value > 0 else "Negative"
        corr_table_data.append([feature, round(corr_value, 3), strength, direction])
    
    corr_df = pd.DataFrame(corr_table_data, 
                           columns=['Feature', 'Correlation with Loss Ratio', 'Strength', 'Direction'])
    print(corr_df.to_markdown(index=False))
    corr_df.to_csv('../reports/tables/loss_ratio_correlations.csv', index=False)
    print("\n✓ Saved to reports/tables/loss_ratio_correlations.csv")
else:
    print("No numeric features found for correlation analysis")

# Table 3: Top Risk Provinces
print("\n" + "="*80)
print("TABLE 3: Top 5 Highest Risk Provinces (Minimum 100 policies)")
print("="*80)

province_stats = df.groupby('province', observed=False).agg({
    'loss_ratio': ['mean', 'count'],
    'totalpremium': 'sum',
    'totalclaims': 'sum'
}).round(4)

# Flatten column names
province_stats.columns = ['mean_loss_ratio', 'policy_count', 'total_premium', 'total_claims']

# Filter for meaningful sample sizes
province_stats = province_stats[province_stats['policy_count'] >= 100]

# Calculate additional metrics
province_stats['avg_premium_per_policy'] = (province_stats['total_premium'] / province_stats['policy_count']).round(2)
province_stats['avg_claims_per_policy'] = (province_stats['total_claims'] / province_stats['policy_count']).round(2)

# Sort and get top 5
top_provinces = province_stats.sort_values('mean_loss_ratio', ascending=False).head(5)

# Create display table
province_display = pd.DataFrame({
    'Province': top_provinces.index,
    'Mean Loss Ratio': top_provinces['mean_loss_ratio'].round(3),
    'Policy Count': top_provinces['policy_count'].astype(int).apply(lambda x: f"{x:,}"),
    'Avg Premium per Policy': top_provinces['avg_premium_per_policy'].apply(lambda x: f"${x:,.2f}"),
    'Avg Claims per Policy': top_provinces['avg_claims_per_policy'].apply(lambda x: f"${x:,.2f}")
})

# Add risk tiers based on loss ratio
risk_tiers = []
for ratio in top_provinces['mean_loss_ratio']:
    if ratio > 0.3:
        risk_tiers.append('🔴 High Risk')
    elif ratio > 0.25:
        risk_tiers.append('🟠 Elevated Risk')
    elif ratio > 0.2:
        risk_tiers.append('🟡 Medium Risk')
    else:
        risk_tiers.append('🟢 Low Risk')

province_display['Risk Tier'] = risk_tiers

print(province_display.to_markdown(index=False))
province_display.to_csv('../reports/tables/top_risk_provinces.csv', index=False)
print("\n✓ Saved to reports/tables/top_risk_provinces.csv")

# Table 4: Vehicle Type Risk Analysis
print("\n" + "="*80)
print("TABLE 4: Vehicle Types by Risk Level (Top 6, Minimum 50 policies)")
print("="*80)

vehicle_stats = df.groupby('vehicletype', observed=False).agg({
    'loss_ratio': ['mean', 'count'],
    'totalpremium': 'sum',
    'totalclaims': 'sum'
}).round(4)

vehicle_stats.columns = ['mean_loss_ratio', 'policy_count', 'total_premium', 'total_claims']

# Filter for meaningful sample sizes
vehicle_stats = vehicle_stats[vehicle_stats['policy_count'] >= 50]

# Sort and get top 6
top_vehicles = vehicle_stats.sort_values('mean_loss_ratio', ascending=False).head(6)

# Create display table
vehicle_display = pd.DataFrame({
    'Vehicle Type': top_vehicles.index,
    'Mean Loss Ratio': top_vehicles['mean_loss_ratio'].round(3),
    'Policy Count': top_vehicles['policy_count'].astype(int).apply(lambda x: f"{x:,}"),
    'Total Premium': top_vehicles['total_premium'].apply(lambda x: f"${x:,.0f}"),
    'Total Claims': top_vehicles['total_claims'].apply(lambda x: f"${x:,.0f}")
})

# Add risk levels
risk_levels = []
for ratio in top_vehicles['mean_loss_ratio']:
    if ratio > 0.4:
        risk_levels.append('🔴 Extreme Risk')
    elif ratio > 0.3:
        risk_levels.append('🟠 High Risk')
    elif ratio > 0.2:
        risk_levels.append('🟡 Medium Risk')
    else:
        risk_levels.append('🟢 Low Risk')

vehicle_display['Risk Level'] = risk_levels

print(vehicle_display.to_markdown(index=False))
vehicle_display.to_csv('../reports/tables/vehicle_type_risk_table.csv', index=False)
print("\n✓ Saved to reports/tables/vehicle_type_risk_table.csv")

# Table 5: Time Series Summary Statistics
print("\n" + "="*80)
print("TABLE 5: Time Series Claims Summary (Oct 2013 - Aug 2015)")
print("="*80)

# Calculate time series statistics if available
if 'time_series_df' in locals() or 'time_series_df' in globals():
    ts_stats = pd.DataFrame({
        'Metric': ['Total Months', 'Average Monthly Claims', 'Minimum Monthly Claims', 
                   'Maximum Monthly Claims', 'Standard Deviation', 'Total Claims Period'],
        'Value': [
            len(time_series_df),
            f"${time_series_df['totalclaims'].mean():,.0f}",
            f"${time_series_df['totalclaims'].min():,.0f}",
            f"${time_series_df['totalclaims'].max():,.0f}",
            f"${time_series_df['totalclaims'].std():,.0f}",
            f"${time_series_df['totalclaims'].sum():,.0f}"
        ]
    })
    print(ts_stats.to_markdown(index=False))
    ts_stats.to_csv('../reports/tables/time_series_summary.csv', index=False)
    print("\n✓ Saved to reports/tables/time_series_summary.csv")
else:
    print("Time series data not available for summary statistics")

print("\n" + "="*80)
print("✅ GENERATION COMPLETE!")
print("="*80)
print("\n📊 VISUALIZATIONS GENERATED:")
print("  1. province_risk_dist.png       - Loss Ratio by Province (Top 10)")
print("  2. vehicle_type_risk.png        - Loss Ratio by Vehicle Type (Top 8)")
print("  3. correlation_heatmap.png      - Feature Correlation Matrix")
print("  4. claims_time_series.png       - Monthly Claims Time Series")
print("\n📋 TABLES GENERATED:")
print("  1. descriptive_stats.csv        - Core financial metrics")
print("  2. loss_ratio_correlations.csv  - Top feature correlations")
print("  3. top_risk_provinces.csv       - Highest risk provinces")
print("  4. vehicle_type_risk_table.csv  - Vehicle type risk analysis")
print("  5. time_series_summary.csv      - Time series statistics")
print("\n📝 INSTRUCTIONS:")
print("  1. Copy the markdown tables above into your report")
print("  2. Reference images in report: ![Description](reports/figures/filename.png)")
print("  3. Update figure numbers in report as needed")
print("  4. All files saved to 'reports/' directory")

Generating all missing visualizations and tables...

1. Generating Province Risk Distribution...
   ✓ Saved to ../reports/figures/province_risk_dist.png

2. Generating Vehicle Type Risk Distribution...
   ✓ Saved to ../reports/figures/vehicle_type_risk.png

3. Generating Correlation Heatmap...
   ✓ Saved to ../reports/figures/correlation_heatmap.png

4. Generating Statistical Tables...

TABLE 1: Descriptive Statistics of Core Financial Metrics
|       |   totalpremium |     totalclaims |    loss_ratio |
|:------|---------------:|----------------:|--------------:|
| count |     1.0001e+06 |      1.0001e+06 |    1.0001e+06 |
| mean  |    61.91       |     64.86       |    0.22       |
| std   |   230.28       |   2384.07       |    7.3        |
| min   |  -782.58       | -12002.4        |  -18.7        |
| 25%   |     0          |      0          |    0          |
| 50%   |     2.18       |      0          |    0          |
| 75%   |    21.93       |      0          |    0          |
| m